<a href="https://colab.research.google.com/github/ritikkumar01111999/data_science/blob/main/taggg_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns

import warnings

import pickle
import time

import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.metrics import hamming_loss
import sklearn.metrics as skm


warnings.filterwarnings("ignore")

In [2]:
from google.colab import files

uploaded = files.upload()

Saving data.csv to data.csv


In [3]:
#data = pd.read_csv("/content/drive/MyDrive/data___/data.csv")
import pandas as pd
data=pd.read_csv('/content/data.csv')
data.head(10)

,Unnamed: 0,tags,description
0,0,licence-needed supervising-job 5-plus-years-ex...,THE COMPANY Employer is a midstream service...
1,1,2-4-years-experience-needed salary full-time-job,ICR Staffing is now accepting resumes for Indu...
2,2,part-time-job,This is a great position for the right person....
3,3,licence-needed,A large multi-specialty health center is expan...
4,4,5-plus-years-experience-needed full-time-job b...,JOB PURPOSE: The Account Director is respon...
5,5,associate-needed 5-plus-years-experience-needed,"At Cottage Health System, our facilities are s..."
6,6,full-time-job ms-or-phd-needed 5-plus-years-ex...,*E Learning Instructional Designer with at lea...
7,7,bs-degree-needed 5-plus-years-experience-needed,Sales and Service Representative SAF-Hollan...
8,8,bs-degree-needed 2-4-years-experience-needed,"SAF-Holland, Inc. is currently seeking an Inte..."
9,9,full-time-job hourly-wage,Immediate opening for full time staffing coord...


In [4]:
data.head()

,Unnamed: 0,tags,description
0,0,licence-needed supervising-job 5-plus-years-ex...,THE COMPANY Employer is a midstream service...
1,1,2-4-years-experience-needed salary full-time-job,ICR Staffing is now accepting resumes for Indu...
2,2,part-time-job,This is a great position for the right person....
3,3,licence-needed,A large multi-specialty health center is expan...
4,4,5-plus-years-experience-needed full-time-job b...,JOB PURPOSE: The Account Director is respon...


In [5]:
data.drop(columns=['Unnamed: 0'], inplace=True)

In [6]:
data.head()

,tags,description
0,licence-needed supervising-job 5-plus-years-ex...,THE COMPANY Employer is a midstream service...
1,2-4-years-experience-needed salary full-time-job,ICR Staffing is now accepting resumes for Indu...
2,part-time-job,This is a great position for the right person....
3,licence-needed,A large multi-specialty health center is expan...
4,5-plus-years-experience-needed full-time-job b...,JOB PURPOSE: The Account Director is respon...


In [7]:
data.shape

(4375, 2)

## Checkpoint

In [8]:
df = data.copy()

df.head()

,tags,description
0,licence-needed supervising-job 5-plus-years-ex...,THE COMPANY Employer is a midstream service...
1,2-4-years-experience-needed salary full-time-job,ICR Staffing is now accepting resumes for Indu...
2,part-time-job,This is a great position for the right person....
3,licence-needed,A large multi-specialty health center is expan...
4,5-plus-years-experience-needed full-time-job b...,JOB PURPOSE: The Account Director is respon...


## Data Cleaning

In [9]:
df.isnull().sum()

tags           871
description      0
dtype: int64

In [10]:
df.dropna(inplace=True)

In [11]:
df.isnull().sum()

tags           0
description    0
dtype: int64

In [12]:
# load stop words
# import nltk
nltk.download('stopwords')
nltk.download('wordnet')
stop_word = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [13]:
def clean(text):
    from nltk.corpus import stopwords

    #     remove urls
    text = re.sub(r'http\S+', " ", text)

    #     remove mentions
    text = re.sub(r'@\w+',' ',text)

    #     remove hastags
    text = re.sub(r'#\w+', ' ', text)

    #     remove digits
    text = re.sub(r'\d+', ' ', text)

    #     remove html tags
    text = re.sub('r<.*?>',' ', text)
    
    #     remove stop words 
    text = text.split()
    text = " ".join([word for word in text if not word in stop_word])
    
      
    return text


In [14]:
df['description'] = df['description'].apply(lambda x: clean(x))

In [15]:
df.head()

,tags,description
0,licence-needed supervising-job 5-plus-years-ex...,THE COMPANY Employer midstream service provide...
1,2-4-years-experience-needed salary full-time-job,ICR Staffing accepting resumes Industrial Main...
2,part-time-job,This great position right person. HEALTHCAREse...
3,licence-needed,A large multi-specialty health center expandin...
4,5-plus-years-experience-needed full-time-job b...,JOB PURPOSE: The Account Director responsible ...


## tags

In [16]:
# split tags in space
df['new_tags'] = df["tags"].apply(lambda x: x.split())

In [17]:
# add all tags
all_tags = [item for sublist in df['new_tags'].values for item in sublist]

In [18]:
len(all_tags)

6880

In [19]:
all_tags

['licence-needed',
 'supervising-job',
 '5-plus-years-experience-needed',
 '2-4-years-experience-needed',
 'salary',
 'full-time-job',
 'part-time-job',
 'licence-needed',
 '5-plus-years-experience-needed',
 'full-time-job',
 'bs-degree-needed',
 'supervising-job',
 'associate-needed',
 '5-plus-years-experience-needed',
 'full-time-job',
 'ms-or-phd-needed',
 '5-plus-years-experience-needed',
 'bs-degree-needed',
 '5-plus-years-experience-needed',
 'bs-degree-needed',
 '2-4-years-experience-needed',
 'full-time-job',
 'hourly-wage',
 'licence-needed',
 '2-4-years-experience-needed',
 'full-time-job',
 'hourly-wage',
 'bs-degree-needed',
 '2-4-years-experience-needed',
 'hourly-wage',
 'bs-degree-needed',
 '2-4-years-experience-needed',
 'full-time-job',
 'salary',
 'bs-degree-needed',
 '2-4-years-experience-needed',
 '5-plus-years-experience-needed',
 'bs-degree-needed',
 'supervising-job',
 '1-year-experience-needed',
 '2-4-years-experience-needed',
 'salary',
 '2-4-years-experience-n

In [20]:
# getting unique set
my_set = set(all_tags)
unique_tags = list(my_set)
len(unique_tags)

12

In [21]:
unique_tags

['hourly-wage',
 'salary',
 'bs-degree-needed',
 '2-4-years-experience-needed',
 'ms-or-phd-needed',
 'supervising-job',
 'part-time-job',
 '1-year-experience-needed',
 'full-time-job',
 'associate-needed',
 '5-plus-years-experience-needed',
 'licence-needed']

In [22]:
# we have all 6800 tag but repeated so after finding unique values we only left with 12 

## Preprocessing

In [23]:

token=ToktokTokenizer()
lemma=WordNetLemmatizer()

def lemitizeWords(text):
    words=token.tokenize(text)
    listLemma=[]
    for w in words:
        x=lemma.lemmatize(w, pos="v")
        listLemma.append(x)
    return ' '.join(map(str, listLemma))

In [24]:
df['description'] = df['description'].apply(
              lambda x: lemitizeWords(x)) 

In [25]:
df['description'].head(10)

0    THE COMPANY Employer midstream service provide...
1    ICR Staffing accept resume Industrial Maintena...
2    This great position right person. HEALTHCAREse...
3    A large multi-specialty health center expand A...
4    JOB PURPOSE : The Account Director responsible...
5    At Cottage Health System , facilities state ar...
6    *E Learning Instructional Designer least + yea...
7    Sales Service Representative SAF-Holland lead ...
8    SAF-Holland , Inc. currently seek Internal Aud...
9    Immediate open full time staff coordinator Hea...
Name: description, dtype: object

## Model Building

In [26]:
df.head()

,tags,description,new_tags
0,licence-needed supervising-job 5-plus-years-ex...,THE COMPANY Employer midstream service provide...,"[licence-needed, supervising-job, 5-plus-years..."
1,2-4-years-experience-needed salary full-time-job,ICR Staffing accept resume Industrial Maintena...,"[2-4-years-experience-needed, salary, full-tim..."
2,part-time-job,This great position right person. HEALTHCAREse...,[part-time-job]
3,licence-needed,A large multi-specialty health center expand A...,[licence-needed]
4,5-plus-years-experience-needed full-time-job b...,JOB PURPOSE : The Account Director responsible...,"[5-plus-years-experience-needed, full-time-job..."


In [27]:
x_train=df['description'][:3000]
x_test=df['description'][3000:]

y_train = df['tags'][:3000]
y_test =  df['tags'][3000:]

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(tokenizer = lambda x: x.split(), binary='true')

In [29]:
multilabel_y_train = vectorizer.fit_transform(y_train)
multilabel_y_test = vectorizer.fit_transform(y_test)

In [30]:
vectorizer.get_feature_names()

['1-year-experience-needed',
 '2-4-years-experience-needed',
 '5-plus-years-experience-needed',
 'associate-needed',
 'bs-degree-needed',
 'full-time-job',
 'hourly-wage',
 'licence-needed',
 'ms-or-phd-needed',
 'part-time-job',
 'salary',
 'supervising-job']

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(smooth_idf=True, norm="l2", \
                             tokenizer = lambda x: x.split(), sublinear_tf=False, ngram_range=(1,3))
x_train_multilabel = vectorizer.fit_transform(x_train)
x_test_multilabel = vectorizer.transform(x_test)

In [32]:
print("Dimensions of train data X:",x_train_multilabel.shape, "Y :",multilabel_y_train.shape)
print("Dimensions of test data X:",x_test_multilabel.shape,"Y:",multilabel_y_test.shape)

Dimensions of train data X: (3000, 879818) Y : (3000, 12)
Dimensions of test data X: (504, 879818) Y: (504, 12)


In [58]:
import os
import string
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [34]:
classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.00001, penalty='l1'), n_jobs=-1)
classifier.fit(x_train_multilabel,multilabel_y_train )
predictions = classifier.predict (x_test_multilabel)
predictions_train = classifier.predict (x_train_multilabel)

In [35]:
print(predictions.toarray())

[[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 1 ... 0 0 1]]


In [36]:
y_true = multilabel_y_test.toarray()
y_logits = predictions.toarray()

In [37]:
def result_check(x, y):
     count = 0
     for i in range(len(x)):
         if x[i] == y[i]:
             count += 1
     result = count / len(x)
     return result

In [38]:
print(result_check(y_true[2],y_logits[2]))

0.6666666666666666


In [56]:
threshold_list = [1,.9,.8,.7,.6,.5]
for i in threshold_list:
  print(f'for threshold {i} accuracy is = ',subset_accuracy(y_true,y_logits,threshold=i))

for threshold 1 accuracy is =  0.1349206349206349
for threshold 0.9 accuracy is =  0.49007936507936506
for threshold 0.8 accuracy is =  0.8035714285714286
for threshold 0.7 accuracy is =  0.9484126984126984
for threshold 0.6 accuracy is =  0.9880952380952381
for threshold 0.5 accuracy is =  1.0


In [57]:
def subset_accuracy(y_true,y_predict,threshold):
  count = 0
  for j in range(len(y_true)):
    if result_check(y_true[j] , y_predict[j]) >= threshold:
      count = count + 1
  accuracy = count / len(y_true)
  return accuracy



In [48]:
import sklearn.metrics as metrics
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
print("train Accuracy :",metrics.accuracy_score(multilabel_y_train, predictions_train))
print("test Accuracy :",metrics.accuracy_score(multilabel_y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(multilabel_y_test,predictions))


precision = precision_score(multilabel_y_test, predictions, average='micro')
recall = recall_score(multilabel_y_test, predictions, average='micro')
f1 = f1_score(multilabel_y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(multilabel_y_test, predictions, average='macro')
recall = recall_score(multilabel_y_test, predictions, average='macro')
f1 = f1_score(multilabel_y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

train Accuracy : 0.9243333333333333
test Accuracy : 0.1984126984126984
Hamming loss  0.12698412698412698
Micro-average quality numbers
Precision: 0.6167, Recall: 0.5675, F1-measure: 0.5911
Macro-average quality numbers
Precision: 0.6190, Recall: 0.5004, F1-measure: 0.5329


In [61]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier

In [62]:
classifier = OneVsRestClassifier(LinearSVC(verbose=0))
classifier.fit(x_train_multilabel,multilabel_y_train )
predictions = classifier.predict (x_test_multilabel)
predictions_train = classifier.predict (x_train_multilabel)

In [63]:
print(predictions.toarray())

[[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 1 ... 0 0 1]]


In [64]:
y_true = multilabel_y_test.toarray()
y_logits = predictions.toarray()

In [65]:
def result_check(x, y):
     count = 0
     for i in range(len(x)):
         if x[i] == y[i]:
             count += 1
     result = count / len(x)
     return result

In [66]:
print(result_check(y_true[2],y_logits[2]))

0.75


In [67]:
def subset_accuracy(y_true,y_predict,threshold):
  count = 0
  for j in range(len(y_true)):
    if result_check(y_true[j] , y_predict[j]) >= threshold:
      count = count + 1
  accuracy = count / len(y_true)
  return accuracy



In [68]:
threshold_list = [1,.9,.8,.7,.6,.5]
for i in threshold_list:
  print(f'for threshold {i} accuracy is = ',subset_accuracy(y_true,y_logits,threshold=i))

for threshold 1 accuracy is =  0.1349206349206349
for threshold 0.9 accuracy is =  0.49007936507936506
for threshold 0.8 accuracy is =  0.8035714285714286
for threshold 0.7 accuracy is =  0.9484126984126984
for threshold 0.6 accuracy is =  0.9880952380952381
for threshold 0.5 accuracy is =  1.0


In [69]:
import sklearn.metrics as metrics
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
print("train Accuracy :",metrics.accuracy_score(multilabel_y_train, predictions_train))
print("test Accuracy :",metrics.accuracy_score(multilabel_y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(multilabel_y_test,predictions))


precision = precision_score(multilabel_y_test, predictions, average='micro')
recall = recall_score(multilabel_y_test, predictions, average='micro')
f1 = f1_score(multilabel_y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(multilabel_y_test, predictions, average='macro')
recall = recall_score(multilabel_y_test, predictions, average='macro')
f1 = f1_score(multilabel_y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

train Accuracy : 1.0
test Accuracy : 0.1349206349206349
Hamming loss  0.13640873015873015
Micro-average quality numbers
Precision: 0.6539, Recall: 0.3323, F1-measure: 0.4407
Macro-average quality numbers
Precision: 0.5864, Recall: 0.2622, F1-measure: 0.3260


In [70]:
classifier = OneVsRestClassifier(MultinomialNB())
classifier.fit(x_train_multilabel,multilabel_y_train )
predictions = classifier.predict (x_test_multilabel)
predictions_train = classifier.predict (x_train_multilabel)

In [71]:
print(predictions.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [72]:
y_true = multilabel_y_test.toarray()
y_logits = predictions.toarray()

In [73]:
def result_check(x, y):
     count = 0
     for i in range(len(x)):
         if x[i] == y[i]:
             count += 1
     result = count / len(x)
     return result

In [80]:
print(result_check(y_true[3],y_logits[3]))

0.8333333333333334


In [75]:
def subset_accuracy(y_true,y_predict,threshold):
  count = 0
  for j in range(len(y_true)):
    if result_check(y_true[j] , y_predict[j]) >= threshold:
      count = count + 1
  accuracy = count / len(y_true)
  return accuracy



In [82]:
print(subset_accuracy(y_true[3],y_logits[3],threshold))

NameError: ignored

In [76]:
threshold_list = [1,.9,.8,.7,.6,.5]
for i in threshold_list:
  print(f'for threshold {i} accuracy is = ',subset_accuracy(y_true,y_logits,threshold=i))

for threshold 1 accuracy is =  0.0
for threshold 0.9 accuracy is =  0.36904761904761907
for threshold 0.8 accuracy is =  0.7579365079365079
for threshold 0.7 accuracy is =  0.9404761904761905
for threshold 0.6 accuracy is =  0.9920634920634921
for threshold 0.5 accuracy is =  1.0


In [77]:

import sklearn.metrics as metrics
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
print("train Accuracy :",metrics.accuracy_score(multilabel_y_train, predictions_train))
print("test Accuracy :",metrics.accuracy_score(multilabel_y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(multilabel_y_test,predictions))


precision = precision_score(multilabel_y_test, predictions, average='micro')
recall = recall_score(multilabel_y_test, predictions, average='micro')
f1 = f1_score(multilabel_y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(multilabel_y_test, predictions, average='macro')
recall = recall_score(multilabel_y_test, predictions, average='macro')
f1 = f1_score(multilabel_y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

train Accuracy : 0.0006666666666666666
test Accuracy : 0.0
Hamming loss  0.16170634920634921
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
Macro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
